In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AdamW
import torch

In [ ]:
raw_datasets = load_dataset("glue","mrpc")

In [ ]:
checkpoint = 'bert-base-cased'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenizer_fn(examples):
    return tokenizer(examples['sentence1'],examples['sentence2'], truncation = True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenizer_fn, batched = True)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(['sentence1','sentence2','idx'])
tokenized_datasets

In [ ]:
tokenized_datasets = tokenized_datasets.rename_column('label','labels')

In [ ]:
tokenized_datasets.set_format('torch')

In [ ]:
tokenized_datasets


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_dataloader = DataLoader(tokenized_datasets['train'], shuffle = True, batch_size = 8, collate_fn = data_collator)

In [ ]:
eval_dataloader = DataLoader(tokenized_datasets['validation'], batch_size = 8, collate_fn= data_collator)

In [ ]:
flag = 0
data_dict=[]
for i in train_dataloader:
    if flag > 3:
        break
    
    data_dict.append([{k:v.shape for k,v in i.items()}])
    flag = flag + 1

data_dict

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)

In [ ]:
outputs = model(**i)

In [ ]:
print(outputs.loss)
print(outputs.logits.shape)
print(outputs.logits)


In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

loss = outputs.loss
loss.backward()
optimizer.step()
optimizer.zero_grad()

In [ ]:
from transformers import get_scheduler

In [ ]:
len(train_dataloader)

len(train_dataloader) = num_rows_train_dataset / batch_size_8 = 3668/8 = 458.5 
num_training_steps = 458.5 * 3 = 1377  


In [ ]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)

In [ ]:
num_training_steps

In [ ]:
lr_scheduler = get_scheduler("linear", optimizer = optimizer, num_warmup_steps = 0, num_training_steps = num_training_steps)

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device('cpu')

In [ ]:
model.to(device)
print(device)

In [ ]:
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

In [ ]:
model.train()

In [ ]:
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k:v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        
        #optimizer.add_param_groupstep()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
pip install evaluate

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

In [ ]:
import numpy as np
from sklearn.metrics import classification_report
preds = np.argmax(predictions.predictions, axis=-1)
print(classification_report(y_true=predictions.label_ids, y_pred=preds))